# Mecanismo de clasificación por matrices salvajes

## 1. Introducción

### Clasificación por KNN y sus limitantes
El algoritmo KNN se basa en el cálculo de distancia entre las secuencias o "disimilitud" utilizando una única matriz de distancia para cuantificar la diferencia entre cada par de secuencias de consulta (**query**) y referencia (**reference**).

La naturaleza discreta de los atributos en consideración (bases de ADN), implica que las instancias de referencia se disponen en forma escalonada o en "orbitas" en torno a una instancia de consulta dada $q$.

![Orbitas](Figuras/orbitals_knn.png)

Esto genera una complicación en la selección de $K$ vecinos característica del algoritmo *KNN*: Dado que todas las instancias que se encuentran en un mismo orbital son equidistantes a $q$, es posible que un número de instancias de referencia mayor a $K$ sean igualmente próximos a $q$. En este caso, atenerse al valor de $K$ original implica seleccionar arbitrariamente un subconjunto de vecinos y excluír a otros. Resulta deseable, por lo tanto, flexibilizar el criterio de selección de vecinos.

En Graboid, el agrumento *criterion* permite al usuario seleccionar el criterio empleado para delimitar el conjunto de vecinos más cercanos. Los dos criterios disponibles son:

* Delimitación por **orbital**: El conjunto de vecinos más cercanos se construye tomando todas las instancias presentes en los $K$ orbitales más cercanos a $q$.
* Delimitación por **vecinos**: Se incluyen orbitales hasta el que contiene al $K$-ésimo vecino más cercano a $q$

Independientemente del criterio elegido, el número final de vecinos empleado para cada instancia dependerá de la cantidad de vecinos en las inmediaciones de $q$ en el espacio muestral.

El uso de una misma matriz de distancia para cuantificar la diferencia entre todos los sitios de dos secuencias presenta una limitante en el sentido de que se da la misma importancia a todos los sitios. Todas las coincidencias son igualmente valoradas. Bajo este esquema, un sitio invariable a nivel de todas las secuencias de referencia es igual de relevante que un sitio que presenta un valor único en un taxón de interés $t$.

### El problema de clasificación

Los elementos del problema son:
* Una instancia de consulta $q$
* Un alineamiento de referencia $R$ conformado por secuencias de clasificación $r_{t}$, $t \in T$ conocida
* Un conjunto $T$ de taxones representados en $R$

El algoritmo KNN busca asignar una clasificación $q_t \in T$ a $q$, la cual se determina en base a la clasificación mayoritaria entre las $K$ secuencias de referencia más cercanas (menos disímiles) a $q$.

La distancia entre secuencias $d(q,r)$ se calcula de la forma: $$\sum_{i=1}^n M[q_i, r_i]$$
Siendo $n$ el número de sitios considerados. La función $d(q,r)$ emplea una matriz de distancia $M$ para cuantificar el grado de diferencia (o similitud) entre secuencias.

![Distancias](Figuras/distance_calc.png)

El hecho de utilizar una misma matriz para caracterizar todos los sitios del alineamiento presenta una limitación en el sentido de que supone que todos los sitios son igualmente relevantes para identificar todos los taxones. Adicionalmente, todos los sitios coincidentes aportan el mismo indicio de similitud entre secuencias.

A modo de ilustración: Supóngase un taxón $t \in T$ y dos sitios del alineamiento $A$ y $B$. Cada sitio presenta un valor único en todos los representantes de $t$:
$$R_A = A_t \forall s \in t$$
$$R_B = B_t \forall s \in t$$
El valor $A_t$ también se observa en el sitio $A$ en secuencias ajenas a $t$, mientras que $B_t$ solo se observa en los representantes de $t$. El sitio $B$ es definitivamente más útil para detectar la presencia de $t$, sin embargo, se le da la misma importancia que al sitio $A$. Esto puede representar un problema cuando la cantidad de sitios del tipo de $A$ es significativamente mayor que la del tipo $B$, dado que fomenta la inclusión de instancias pertenecientes a otros taxones junto con los representantes de $t$.

Resulta necesario modificar el algoritmo de manera que las coincidencias informativas de la presencia de un taxón dado se tomen en mayor consideración que las coincidencias no informativas.

### Matrices salvajes

Modificaciones introducidas al algoritmo:

* Reemplazar la medida de **distancia** por **señal**. En lugar de utilizar la disimilitud de secuencias, se consideran las coincidencias en sitios relevantes
* Reemplazar el criterio de **vecinos más cercanos** por el de **clase con señal más fuerte**. En lugar de calcular distancias a todas las instancias de referencia, se calcula la señal acumulada por cada uno de los taxones representados en $R$. La clase $t_q$ asignada a $q$ es aquella con la señal más fuerte (normalizada para el número de representantes)
* Redefinir la matriz de distancia $M$ de forma $[5, 5]$ como una matriz tridimensional de forma $[5,5,n]$, en la que cada "capa" (submatriz de dimensiones $[5, 5]$ corresponde a un sitio del alineamiento.
* Sustituir la función de distancia $d(q, r)$ por una función de señal
$$s(q, r) = \sum_{i=1}^n M[q_i, r_i, i]$$
Esta función es similar a $d(q,r)$, pero fue modificada para utilizar la nueva matriz $M$. Para cada sitio de las secuencias se utiliza una capa diferente de $M$. Esto permite considerar a cada sitio por separado sin incrementar el número de operaciones.

#### Características de la nueva matriz
El propósito de la matriz *salvaje* es habilitar la consideración por separado de cada uno de los sitios del alineamiento. El funcionamiento de la matriz es el siguiente:
Dado un taxón $t \in T$, suponer que existe un sitio $A$ del alineamiento $R$ que contiene un valor $A_t$ que solo se observa en instancias de $t$. Podemos decir que, dados los datos, la presencia del valor $A_t$ en el sitio $A$ de cualquier secuencia $q$ es evidencia de que dicha secuencia pertenece a $t$.

Es posible representar esta hipótesis en una matriz de dimensión $[5, 5]$ cuyos valores son $0$ en todas las celdas excepto en la posición $[B, B]$, en la cual presenta un valor de señal arbitrario. De esta manera, solo se emite señal para $t$ cuando $Q_A = A_t$ y $R_A = A_t$.

Para cada sitio con valores informativos para identificar algún taxón se construyen matrices similares a esta. El resultado final es una matriz de dimensiones $[5,5,n]$

#### Calculo de señal
En el nuevo procedimiento, se define un vector de señales $S$ con $|T|$ elementos, correspondientes a los diferentes taxones representados en el alineamiento de referencia.

Cada secuencia de consulta $r$ es evaluada contra la secuencia $q$ utilizando la función $s(q, r, M)$. La señal resultante se adiciona al elemento correspondiente al taxón de $r$, $t_r$ en $S$.

Luego de evaluar todas las secuencias de referencia, la clase asignada a la instancia de consulta se define como:
$$t_q = argmax(S)$$

#### Tipificación de sitios

Para construir la matriz de señales es necesario identificar los sitios con valores que permiten distinguir algún taxón $t$ del resto de $T$. Se define $V_s$ como el conjunto de valores observados en un sitio $s$ cualquiera. Dado un taxón $t$, llamamos $V_t^s$ al conjunto de valores observados en $s$ para los representantes de $t$, y $V_{!t}^s$ al conjunto de valores observados en todas las secuencias ajenas a $t$ en $s$. En función de estos conjuntos, el sitio $s$ se puede tipificar con respecto a $t$ como:

##### Tipo 1
Ninguno de los valores observados en $t$ se encuentra presente en el resto de las secuencias:
$$V_t^s \cap V_{!t}^s = \emptyset$$
En este caso, la ocurrencia de cualquier valor $v \in V_t^s$ es un indicio inequívoco de la presencia de $t$

##### Tipo 2
El taxón $t$ presenta al menos un valor típico en $s$, pero algunos de sus representantes presentan valores compartidos con el resto del alineamiento:
$$V_t^s \cap V_{!t}^s \ne \emptyset$$
$$V_t^s \not\subset V_{!t}^s$$
En un sitio de este tipo, es posible identificar algunos de los representantes de $t$ con los valores de $V_t^s$ que no se encuentren en el resto del alineamiento. Llamamos a esto una *señal parcial*

##### Tipo 3
El taxón $t$ presenta un único valor en $s$. Dicho valor se encuentra presente en algunas secuencias ajenas a $t$ (no todas).
$$|V_t^s| = 1$$
$$V_t^s \subset V_{!t}^s$$
Un sitio $s$ de tipo 3 permite diferenciar al taxón $t$ de las secuencias del alineamiento que no comparten el valor único presente en $V_t^s$.

##### Tipo 4
El taxón $t$ presenta múltiples valores, ninguno de los cuales es característico de $t$:
$$|V_t^s| > 1$$
$$V_t^s \subset V_{!t}^s$$
Los sitios de este tipo son los que aportan la menor cantidad de información, ya que al mismo tiempo que subdivide el taxón, ninguno de los subgrupos generados se diferencia del resto del alineamiento.

#### Construcción de la matriz salvaje

La ventaja de la matriz salvaje es que toda la información necesaria para identificar taxones dados los datos de referencia es contenida en un espacio de tamaño $[5,5,n]$.

Al comenzar la construcción, la matriz salvaje se inicializa como una matriz cero
$$M = 0_{5,5,n}$$

El siguiente paso consiste en tipificar los sitios para todos los taxones presentes en el alineamiento, produciendo una tabla $P$ de dimensiones $|T| \times n$, en la que el tipo de cada sitio con respecto a cada taxón se indica con un número del 1 al 4.

A continuación se define la señal emitida por cada valor en cada sitio. En teoría, un único sitio de tipo 1 es suficiente para determinar la presencia de cualquier taxón. Sin embargo, existen dos motivos por lo que resulta deseable incorporar sitios de tipo 2 y tipo 3 en la hipótesis:

1) Es posible encontrar taxones que no presenten sitios de tipo 1 en el segmento de secuencia abarcado por el alineamiento
2) La hipótesis trabaja siempre en el marco de los datos *disponibles*. Un valor tipo 1 es suficiente para identificar un taxón dentro del conjunto de entrenamiento pero es imposible afirmar que no existen taxones que presentan alguno de los valores de $V_t^s$ que todavía no han sido catalogados en la base de datos de la que se obtuvieron los datos de entrenamiento. Incorporar la mayor cantidad de sitios aumenta la robustez de la hipótesis contra esta eventualidad, a pesar de que la señal de los sitios adicionales puede ser de menor calidad que la de los sitios tipo 1.

El proceso de poblamiento de valores en la matriz $M$ es el siguiente:
Para cada taxón $t$:

##### Sitios tipo 1
Se define $S_1^t$ como el conjunto de sitios tipo 1 con respecto a $t$. Para cada sitio $s \in S_1^t$, se actualiza la matriz $M$ de la forma:
$$M[v,v,s] = i \ \forall \ v \ \in V_t^s$$
La variable $i$ corresponde al valor de señal asignado a los valores típicos observados en sitios tipo 1 y tipo 2. Cuando una secuencia de consulta $q$ y una secuencia de referencia $r$ perteneciente a $t$ presentan el mismo valor perteneciente a $V_t^s$, la señal de $t$ para la secuencia $q$ se incrementa en $i$.

##### Sitios con valores típicos no distintivos
Dado un sitio $s \in S$ que presenta un valor $s_\tau$ cualquiera, siendo $R_\tau$ el conjuto de secuencias para las que $s = s_\tau$. Se considera que $s_\tau$ es un *valor típico no distintivo* (VTND) de $t$ si:
$$R_t \cap R_\tau \cap R_{!t} \ne \emptyset$$
$$R_{!t} \not\subset R_\tau$$
$$R_t \cap R_{!t} = \emptyset\ \forall\ v \in s;\ v \ne \tau$$
$s_\tau$ se observa en los representantes de $t$ y en *parte* del resto del alineamiento, dicho de otra manera, existe parte de $R_{!t}$ que es rechazada por la presencia de $s_\tau$. Adicionalmente, $\tau$ es el único valor de $s$ que se observa tanto en $R_t$ y $R_{!t}$.

**Nota:** Es posible que $R_t$ presente valores adicionales a $s_\tau$ en $s$. Si ninguno de estos valores se observa en $R_{!t}$, $s$ es un sitio de **tipo 2** para $t$. En caso contrario, $s$ es un sitio de **tipo 4** para $t$.

Si bien los sitios con VTND no perimiten detectar la presencia de $t$ individualmente, un conjunto de estos puede ser utilizado para construír una *señal compuesta*. Esto consiste consiste en identificar un conjunto de sitios con VTND para $t$ que permitan rechazar la presencia del mayor número de secuencias de $R_{!t}$.

* Se busca definir un conjunto de sitios $\nu_t = \{\nu_1, \nu_2,...\}$ que incluya todos los sitios que contienen la señal compuesta para $t$
* Cada sitio $\nu_i \in \nu_t$ contiene un VTND $v_i$ tal que el conjunto $R_i$ de secuencias para las que $s_i = v_i$ incluye a $R_t$: $$R_t \cup R_i \ne \emptyset\ \forall\ \nu_i \in \nu_t$$
* La intersección de todas las secuencias en las que que $\nu_i = v_i$ para todos los sitios del conjunto $\nu_t$ contiene únicamente secuencias pertenecientes al taxón $t$: $$\bigcap\limits_{i=1}^{|\nu_t|}\nu_i = v_i \subseteq R_t$$

La intersección de todas secuencias en las que todos los sitios de $S_{c}^{t}$ presentan su valor VTND es un subconjunto de $t$

<img src="Figuras/composite_signal.png" width="500"/>
En la figura anterior, se utiliza el conjunto de sitios $A$, $B$, $C$ y $D$ para identificar la presencia de $t$. Cada uno de los sitios contiene un valor: $a_t$, $b_t$, $c_t$ y $d_t$, el cual es un VTND de $t$. Para cada sitio, el subconjunto de secuencias que contienen el VTND de $t$ incluye secuencias ajenas a $t$. Sin embargo, el unico conjunto que contiene la intersección en la que todos los sitios seleccionados muestra el VTND es el de los representantes de $t$.

### Tipificación de sitios variables

Para resolver la carencia del método empleado por Graboid hasta ahora, es necesario determinar cual es la importancia relativa de los diferentes sitios.

Para continuar con el ejemplo de la parte anterior, supongamos un alineamiento de referencia *R* conformado por secuencias de múltiples taxones pertenecientes al conjunto *T*. 

Sustituto del método de diferencia sans representantes. Omitir cálculo de frecuencias y entropías, tipificación se realiza evaluando cambios en el conteo de valores al remover representantes de un taxón, para cada sitio evaluar en cada taxón:
* Cantidad de valores cambiados
* Cantidad de nuevos ceros

Tipificación:

**Tipo 1:** Nuevos ceros > 0 & Nuevos ceros == valores cambiados

**Tipo 2:** Nuevos ceros > 0 & valores cambiados == (Nuevos ceros + 1)

**Tipo 3:** Nuevos ceros == 0 & valores cambiados == 1

**Tipo 4:** Nuevos ceros == 0 & valores cambiados > 1

In [1]:
# Load modules
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import shutil
import sys
import warnings

# ignore warning calls
warnings.filterwarnings('ignore')
# Make files in the Graboid dir importable
sys.path.append(os.path.dirname('/home/hernan/PROYECTOS/Graboid'))

In [2]:
# Load Graboid modules & test data
from Graboid import classification_main_refactory as cls_main
from Graboid.preprocess import feature_selection as fsele# Load data
db_dir = 'database_NCBI'
query_file = 'g12_low_20K-reads.fa'
out_dir = 'salida_pruebas_abril'
shutil.rmtree(out_dir)
evalue=0.0005
dropoff=0.05
min_height=0.1
min_width=2
min_cov=.95
max_unk_thresh=.2
threads=2

# build working directory
calibration_dir, classif_dir, query_dir, warn_dir, metadata = cls_main.make_working_dir(out_dir)

# set graboid database
ref_taxonomy_file, ref_lineages_file, ref_names_file, guide_db, ref_map_file, ref_map_acc_file = cls_main.database.check_database(db_dir)

# set query
qry_map_file, qry_acc_file, qry_rows, qry_ncols = cls_main.set_query(query_file, guide_db, query_dir, evalue=evalue, dropoff=dropoff, min_height=min_height, min_width=min_width, threads=threads)

# Data holder object... holds data (query & reference alignments, taxonomic data)
data_holder = cls_main.ClassifyDataHolder()
data_holder.load_qry_data(qry_map_file, qry_acc_file)
data_holder.load_ref_data(ref_map_file, ref_map_acc_file, ref_taxonomy_file, ref_lineages_file, ref_names_file)
data_holder.filter_data(min_cov)
data_holder.collapse_reference(max_unk_thresh)
print(data_holder.__dict__.keys())

Retrieving sequences...
Building matrix...
dict_keys(['qry_map', 'qry_bounds', 'qry_acc', 'qry_coverage', 'qry_coverage_norm', 'ref_map', 'ref_acc', 'ref_coverage', 'ref_coverage_norm', 'ref_taxids', 'lineage_tab', 'names_tab', 'filtered_sites', 'valid_refs', 'ref_map_collapsed', 'ref_branches', 'ref_tax_collapsed'])


In [3]:
# count number of sequences, number of sites & list ranks present in the alignment
n_seqs = data_holder.ref_map_collapsed.shape[0]
aln_len = data_holder.ref_map_collapsed.shape[1]
db_ranks = 'phylum class order family genus species'.split()
print(f'N seqs (collapsed): {n_seqs}')
print(f'Alignment length: {aln_len}')

N seqs (collapsed): 1411
Alignment length: 274


In [4]:
# build table specifying each TaxId's rank (this should come with the reference data tables)

ranks_tab = pd.Series('', index=data_holder.lineage_tab.index)

for col in data_holder.lineage_tab.columns:
    non_0 = data_holder.lineage_tab[col].loc[lambda x : x > 0].index
    ranks_tab.loc[non_0] = col
ranks_tab = ranks_tab.to_frame(name='Rank')
ranks_tab.value_counts('Rank')# one-hot-encoding the matrix GREATLY speeds up the counting process

Rank
species    7007
genus       898
family      207
order        18
class         2
phylum        1
Name: count, dtype: int64

In [5]:
# one-hot-encode alignment matirx
def encode_alignment(alignment_matrix):
    encoded = np.stack([alignment_matrix == 1,
                               alignment_matrix == 2,
                               alignment_matrix == 3,
                               alignment_matrix == 4], axis=2)
    return encoded

# get 
def get_invariable_sites(count_matrix):
    return (count_matrix != 0).sum(axis=1) == 1
alignment_encoded = encode_alignment(data_holder.ref_map_collapsed)

In [6]:
# define a function to get value counts per site
def count_values(matrix):
    return matrix.sum(axis=0)
    
count_values(alignment_encoded)

array([[   3,    0, 1374,    8],
       [   4,    0, 1382,    0],
       [   2,    1, 1383,    1],
       ...,
       [ 266, 1095,    1,    7],
       [   1,    4,    0, 1363],
       [   5,    0, 1362,    0]])

### Conteo de valores
Se cuentan los valores por sitio para todo el alineamiento y en particular para cada taxón. La variable *count_diff* indica la diferencia de conteos al eliminar los representantes de un taxón.

In [7]:
# get value counts per site per taxon

# get the taxon of each row in the alignment for each rank (this should be included in the data holder)
lineage_flat = pd.concat(data_holder.ref_lineage_collapsed[rk] for rk in data_holder.ref_lineage_collapsed.columns).to_frame(name='TaxId').reset_index(names='idx')
lineage_flat = lineage_flat.query('TaxId != 0')

taxa = [] # keep tabs on which tax goes in which row
tax_counts = []
for tax, subtab in lineage_flat.groupby('TaxId'):
    print(f'Counting for tax {tax}')
    taxa.append(tax)
    tax_counts.append(count_values(alignment_encoded[subtab.idx.values]))
tax_counts = np.array(tax_counts)
full_count = count_values(alignment_encoded)
count_diff = full_count - tax_counts

Counting for tax 6231
Counting for tax 6236
Counting for tax 6237
Counting for tax 6239
Counting for tax 6243
Counting for tax 6244
Counting for tax 6246
Counting for tax 6247
Counting for tax 6248
Counting for tax 6250
Counting for tax 6254
Counting for tax 6256
Counting for tax 6276
Counting for tax 6277
Counting for tax 6286
Counting for tax 6287
Counting for tax 6296
Counting for tax 6301
Counting for tax 6303
Counting for tax 6304
Counting for tax 6305
Counting for tax 6306
Counting for tax 6324
Counting for tax 6325
Counting for tax 6329
Counting for tax 6332
Counting for tax 6333
Counting for tax 6334
Counting for tax 6335
Counting for tax 6336
Counting for tax 13657
Counting for tax 13658
Counting for tax 27830
Counting for tax 27838
Counting for tax 27840
Counting for tax 29166
Counting for tax 29169
Counting for tax 29170
Counting for tax 31237
Counting for tax 31239
Counting for tax 31242
Counting for tax 31243
Counting for tax 33253
Counting for tax 33278
Counting for tax 3

### Construir matriz binaria por taxón
Para cada taxón construír una matriz binaria de forma *n_sites* x *4*. Esta matriz indica los valores que presenta el taxón en cada uno de los sitios. Los sitios variables en el taxón presentarán múltiples valores verdaderos en la columna correspondiente.

El motivo de estas matrices es reflejar la información contenida en cada sitio sin introducir un sesgo en la señal por los sitios invariables que aparecen múltiples veces cuando un taxón presenta múltiples representantes.

In [8]:
# Build flat binary matrix
flat_bin_mat = tax_counts != 0
flat_bin_mat.shape

(1294, 274, 4)

### Cambio de valores
Determinar presencia de **nuevos ceros** y **valores cambiados**

In [9]:
# get changed values
changed_vals = (count_diff != full_count).sum(axis=2)

# get new zeros
new_zeros = ((full_count != 0) & (count_diff == 0)).sum(axis=2)

### Tipificación
Construír una tabla de tipificación, indicando para cada taxón, el tipo de cada sitio.

También se construye una tabla sumario en la que se indica cuantos sitios de cada tipo hay en cada taxón

In [10]:
# get types

# type 1 
type1 = ((new_zeros > 0) & (new_zeros == changed_vals)).astype(int)

# type 2
type2 = ((new_zeros > 0) & (changed_vals > new_zeros)).astype(int)

# type 3
type3 = ((new_zeros == 0) & (changed_vals == 1)).astype(int)

# type 4
type4 = ((new_zeros == 0) & (changed_vals > 1)).astype(int)

# build type matrix and table
type_matrix = type1 + type2*2 + type3*3 + type4*4
type_tab = pd.DataFrame(type_matrix, index=pd.Series(taxa, name='Taxon'))

# summarize type table
type_tab_summ = np.array([type1.sum(axis=1),
                          type2.sum(axis=1),
                          type3.sum(axis=1),
                          type4.sum(axis=1)]).T
type_tab_summ = pd.DataFrame(type_tab_summ, columns = pd.Series([1,2,3,4], name='Site_Type'), index=pd.Series(taxa, name='Taxon'))
type_tab_summ[0] = type_tab.shape[1] - type_tab_summ.sum(axis=1)

In [11]:
type_tab

,0,1,2,3,4,5,6,7,8,9,...,264,265,266,267,268,269,270,271,272,273
Taxon,,,,,,,,,,,,,,,,,,,,,
6231,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6236,4,4,2,4,4,2,4,2,2,4,...,4,2,2,2,4,2,2,2,4,4
6237,3,3,3,3,3,3,3,3,3,3,...,4,3,3,3,3,3,3,4,3,3
6239,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6243,3,4,2,3,4,3,3,3,3,4,...,4,4,4,2,3,3,4,4,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3050674,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3060280,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3071791,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [12]:
type_tab_summ

Site_Type,1,2,3,4,0
Taxon,,,,,
6231,274,0,0,0,0
6236,0,159,9,106,0
6237,0,2,257,15,0
6239,1,0,273,0,0
6243,0,8,187,79,0
...,...,...,...,...,...
3050674,1,0,273,0,0
3060280,0,0,274,0,0
3071791,0,0,274,0,0


## Construcción de matrices de señal
Para cada rango se construye una matriz tridimensional de forma *5* x *5* x *n_sitios*. Cada matriz de *5* x *5* se utiliza para emitir señal a partir de los valores encontrados en un sitio del alineamiemto.

![Matrices salvajes](Figuras/savage_matrices.png)

### Paso 1. Seleccionar la subtabla de tipos correspondiente al rango (order)
Queremos constuir una matriz salvaje por rango taxonómico. Además, queremos seleccionar los sitios que permitan distinguir los taxones de un rango específico en el contexto del rango inmediatamente superior. Es decir, si estamos trabajando a nivel de *orden* y el rango inmediatamente superior es *clase*, queremos distinguir las secuencias de un orden dado de las de los demás ordenes en la misma clase.

### Paso 2. Seleccionar sitios
Para cada taxón *t* de la subtabla de tipos, seleccionamos los sitios que nor permitan identificar la señal de dicho taxón.
* **Sitios tipo 1**: Todos los sitios de este tipo se seleccionan, ya que nos dan la mejor señal
* **Sitios tipo 2 y 3**: Se selecciona la menor cantidad de estos sitios necesarios para resolver la señal del taxón. **¿Qué pasa si el taxón presenta sitios de tipo 1?** Opción de incluir sitios adicionales o no.
* **Sitios tipo 4**: Estos sitios se ignoran, ya que nos dan la señal más "sucia" (no presentan valores únicos en el taxón y ninguno de ellos es típico del taxón).

![Sitios tipo](Figuras/site_types.png)

#### Resolución de señal
Llamamos *resolver la señal* al proceso de selección de sitios que maximiza la señal emitida para el taxón *T* en relación a los demás taxones.

<u>Sitios tipo 1:</u> Un solo sitio **tipo 1** es suficiente para distinguir al taxón *t* de los demás. Todos los sitios de este tipo se incorporan a la señal.

<u>Sitios tipo 2:</u> Es posible que *t* presente un conjunto de sitios de **tipo 2** que contienen una *señal compuesta*: todas las secuencias pertecientes a *t* presentan al menos una observación de un valor típico de *t* en alguno de los sitios **tipo 2**.

Para identificar una *señal compuesta* de sitios **tipo 2**:

Para cada taxón *t*:
* Seleccionar todos los sitios **tipo 2**
* Para cada sitio seleccionado:
   * Identificar secuencias con valores *típicos* (observados sólo en *t*)
* Contar secuencias sin ningún valor *típico*
* Si todas las secuencias de *t* tienen al menos un sitio con valor típico, *t* presenta señal compuesta por sitios **tipo 2**
* Si existen secuencias sin valores típicos, estas secuencias deben resolverse utilizando sitios **tipo 1** o **tipo 3**

<u>Sitios tipo 3:</u> Se selecciona una combinación de sitios **tipo 3** que permita excluír a todos los taxones diferentes de *t*, de manera que la señal resultante para *t* es igual a *# sitios incluídos, y el conjunto de *señales residuales* tiene un valor máximo de *# sitios incluídos - 1*

Comenzamos definiendo una **matriz de compartidos** para cada taxón *t*. Esta matriz tiene forma **# taxones hermanos de *t*** x **# sitios informativos (1, 2 o 3)**, e indica qué taxones hermanos comparten valores con *t* en cada sitio. El proceso de resolución de la señal consiste en seleccionar un conjunto de sitios únicos para *t*. La manera de encontrar este conjunto es seleccionar iterativamente sitios con la menor cantidad de valores compartidos, cada taxón hermano que no comparte valor con *t* en el sitio seleccionado se considera *resuelto*.

Procedimiento:

    1) Seleccionar el sitio con menos observaciones compartidas.
    2) Indicar todos los taxones sin observaciones en el sitio elegido como resueltos y quitarlos de la matriz de residuales (su contenido ya no es importante)
    3) Si no quedan taxones sin resolver, terminar la búsqueda, indicar señal como resuelta.
    4) Volver a contar observaciones, sin las filas de los taxones resueltos
    5) Si no quedan sitios sin recorrer (sitios sin observaciones), terminar la búsqueda, indicar señal como no resuelta y devolver listado de taxones irresolubles.
    6) Volver al inicio. A partir de ahora, si hay un empate de sitios con menos observaciones, seleccionar aquel que tenga la menor cantidad de taxones resueltos para minimizar la señal compartida.

#### Caso de prueba, construcción de matrices a nivel de *orden*
* *rank_present_taxa*: DataFrame con índice *TaxId* y columna *Rank*, indicando el rango taxonómico de cada TaxId presente en el conjunto de referencia
* *orders*: TaxIds de todos los ordenes en el conjunto de referencia
* *orders_types*: Sub tabla indicando el tipo de cada sitio para los ordenes
* *type_counts*: Tabla indicando cuantos sitios de cada tipo presenta cada orden
* *encoded_idx*: Series con índice (con valores repetidos) *TaxId* y valores correspondientes a filas en el alineamiento codificado. Útil para recuperar las filas correspondientes a taxones específicos

In [13]:
# get ranks of present taxa, used to build a savage matrix for each rank
rank_present_taxa = ranks_tab.loc[taxa]

# sample type tab
orders = rank_present_taxa.query('Rank == "order"').index
order_types = type_tab.loc[orders]

# map TaxIds to rows in the enconded matrix
encoded_idx = lineage_flat.set_index('TaxId')['idx'].copy()

# count site types for each taxon
type_counts = pd.DataFrame(columns=pd.Series([1,2,3,4], name='Site_Type'), index=pd.Series(name='Taxon'))
for tax, taxsites in order_types.iterrows():
    type_counts.loc[tax] = taxsites.value_counts()
type_counts = type_counts.fillna(0).astype(int)
type_counts

Site_Type,1,2,3,4
Taxon,,,,
6236,0,159,9,106
6329,0,9,200,65
27838,0,0,252,22
46000,0,5,182,87
57287,0,9,172,93
70219,0,13,187,74
70232,0,1,240,33
70242,0,4,227,43
73930,0,8,228,38


#### Seleccionar sitios **tipo 1**

In [14]:
# build a dictionary to help locate taxa in the encoded matrix
tax_map = {tax:idx for idx, tax in enumerate(taxa)}
order_idxs = np.array([tax_map[o_idx] for o_idx in orders])

orders_sites = flat_bin_mat[order_idxs]
orders_sites.shape

(16, 274, 4)

In [15]:
# identify taxa with type 1 sites
taxa_with_type_1 = type_counts.loc[type_counts[1] > 0].index.values
type_1_sites = {}
type_1_values = {}
for tax in taxa_with_type_1:
    sites = order_types.loc[tax]
    sites = sites.loc[sites == 1].index.values
    vals = []
    for st in sites:
        tax_idx = tax_map[tax]
        tax_vals = flat_bin_mat[tax_idx]
        vals.append(np.array([1,2,3,4])[tax_vals[st]][0])
    type_1_sites[tax] = sites
    type_1_values[tax] = vals

#### Identificar taxones resolvibles por sitios **tipo 2**
* *composite_signal_data*: dictionario que contiene para cada taxón un dataframe en el que se indica qué secuencias se resuelven con cada sitio **tipo 2** identificado
* *site_2_solved*: dataframe en el que se se indica para cada taxón:
    * El número de secuencias internas (*Total_seqs*)
    * La cantidad de secuencias internas resueltas (*Solved_seqs*)
    * Si el taxón es completamente resolvible con los sitios **tipo 2** (*Fully_solved*)
    * Si el taxón no presenta resolución con sitios **tipo 2** (*None_solved*)

In [16]:
# loop over every order

composite_signal_data = {}
site_2_solved = pd.DataFrame(index=orders, columns='Total_seqs Solved_seqs'.split())
for ord_idx, order in enumerate(orders):
    
    # get taxon sequences in the encoded matrix
    tax_seqs_idxs = encoded_idx.loc[[order]]
    tax_seqs = alignment_encoded[tax_seqs_idxs]

    # select type 2 sites
    type_2_sites = order_types.loc[order]
    type_2_sites = type_2_sites.loc[type_2_sites == 2].index.values

    # get other orders
    orders_selector = np.full(len(orders), True)
    orders_selector[ord_idx] = False
    other_orders = orders_sites[orders_selector]
    
    # identify typical values & solved taxa for each type 2 site
    solved_seqs = pd.DataFrame(False, index=tax_seqs_idxs, columns=type_2_sites)
    for site in type_2_sites:
        # typical values, absent in all other orders and present in current order
        typical_values = ~np.any(other_orders[:, site], axis=0) & orders_sites[ord_idx, site]
        
        # get solved sequences
        solved = tax_seqs_idxs[np.any(tax_seqs[:, site][:, typical_values], axis=1)]
        solved_seqs.loc[solved, site] = True
    composite_signal_data[order] = solved_seqs
    site_2_solved.loc[order] = [len(tax_seqs_idxs), np.any(solved_seqs, axis=1).sum()]
site_2_solved['Fully_solved'] = site_2_solved.Total_seqs == site_2_solved.Solved_seqs
site_2_solved['None_solved'] = site_2_solved.Solved_seqs == 0
site_2_solved['Type_2_sites'] = type_counts[2]
site_2_solved

,Total_seqs,Solved_seqs,Fully_solved,None_solved,Type_2_sites
6236,914,424,False,False,159
6329,43,25,False,False,9
27838,16,0,False,True,0
46000,77,5,False,False,5
57287,111,8,False,False,9
70219,42,6,False,False,13
70232,20,1,False,False,1
70242,19,3,False,False,4
73930,40,10,False,False,8
114861,12,1,False,False,1


#### Resolver señal usando sitios **tipo 2** y **tipo 3**

![sig_resolution](Figuras/signal_resolution.png)

* Iterar sobre cada taxón *t*
* Seleccionar todos los sitios **tipo 2** y **tipo 3** para *t*
* Mientras el taxón no esté resuelto o queden sitios sin investigar:
    * Cuantificar el número de observaciones compartidas para cada sitio *s* (cuantas veces se observa el valor *v<sub>t</sub>* fuera de *t*)
    * Seleccionar el sitio *s* con menos observaciones compartidas
    * Eliminar remover a *s* de los sitios sin investigar
    * Eliminar las secuencias con un valor diferente de *v<sub>t</sub>* en *s*

In [18]:
# iterate over every order
for ord in orders:
    # get type 2 & type 3 sites
    sites_23 = order_types.loc[ord]
    sites_23 = sites_23.loc[(sites_23 == 2) | (sites_23 == 3)]

    # select sequences
    selector = np.full(alignment_encoded.shape[0], False)
    selector[encoded_idx.loc[ord]] = True

    # get sequences of current order & remaining sequences
    tax_seqs = alignment_encoded[selector]
    other_seqs = alignment_encoded[~selector]
    
    # select only relevant sites
    tax_seqs = tax_seqs[:,sites_23]
    other_seqs = other_seqs[:, sites_23]

    # drop already solved sequences
    tax_seqs = tax_seqs[~np.any(composite_signal_data[ord], axis=1)]
    
    # get shared values
    tax_values = tax_seqs.any(axis=0)
    
    shared_vals = other_seqs & tax_seqs.any(axis=0)
    break
tax_seqs.shape

(490, 168, 4)

In [168]:
def build_bridge(solvable_seqs, columns=None, solved_seqs=None):
    # recursively build a bridge, select the site that most increases the amount of solved sites with respect to the previous step
    # attempts to build the bridge with the least amount of steps possible
    
    # initialize solved seqs & columns
    if solved_seqs is None:
        solved_seqs = np.full((solvable_seqs.shape[0], 1), False)
    if columns is None:
        columns = np.arange(solvable_seqs.shape[1])

    print(f'{len(columns)} remaining')
    # get new solved sites
    new_solved_seqs = ~solvable_seqs | solved_seqs
    solved_count = new_solved_seqs.sum(axis=0)
    
    # get site that most increases the number of solved sequences
    most_counts = solved_count.max()
    if most_counts == solved_seqs.sum():
        # bridge can't keep growing, theoretically, all sequences have been solved
        return []
    
    # select bridge site
    selected_site = np.argmax(solved_count)
    bridge_sites = [columns[selected_site]]

    # remove selected site from columns # matrix
    selector = np.full(len(columns), True)
    selector[np.argmax(solved_count)] = False
    
    # update solved seqs
    solved_seqs = new_solved_seqs[:, [selected_site]]

    # continue building the bridge
    bridge_sites += build_bridge(solvable_seqs[:, selector], columns[selector], solved_seqs)
    return bridge_sites

def solve_sites_23(shared_values):
    # determine which sequences can be solved using type 2 & 3 sites
    # calculate the shortest way to resolve signal of solvable sites
    # If a path of non-shared values can be followed across all sequences, the signal is solvable.
    
    # check for unsolvable sequences, any sequence with no non-shared values is consudered unsolvable, reported and discarded
    unsolvables = shared_values.all(axis=1)
    solvables = ~unsolvables
    
    print(f'Can solve {solvables.sum()} of {shared_values.shape[0]} sequences')
    # Verify that taxon can be solved
    if solvables.all():
        print('Taxon is fully solvable!')
    if unsolvables.all():
        print('No sequences can be solved!')

    # resolve solvable sequences
    bridge_sites = build_bridge(shared_values[solvables])

    # reinforce bridge???
    return bridge_sites, solvables, unsolvables

In [153]:
# solve tax by sites 2 & 3
# first, build 2d array of shape [other seqs, type2&3 cols]
shared = shared_vals.any(axis=2)
shared.shape

(497, 168)

In [165]:
bridge_sites, solvable, unsolvale = solve_sites_23(shared)

Can solve 12 of 497 sequences
168 remaining
167 remaining
166 remaining


#### Matriz compartida
Esta matriz de forma *n_taxa* x *n_taxa-1* x *n_sites* indica para cada taxón, cuales de los demás taxones presentan valores compartidos en cada sitio.

Usando esta matriz es posible identificar para cada taxón, si lo hubiere, un conjunto de sitios que permita discriminar a los restantes taxones.

In [ ]:
# build shared_matrix
shared_matrix = []
for idx, tax_layer in enumerate(orders_sites):
    inverse_idxs = np.full(orders_sites.shape[0], True)
    inverse_idxs[idx] = False
    brother_layers = orders_sites[inverse_idxs]
    shared_matrix.append((tax_layer & brother_layers).sum(axis=2) != 0)
shared_matrix = np.array(shared_matrix)
shared_matrix.shape
# shared matrix shape is: [#taxa, #taxa-1, #sites]

#### Taxones resolvibles
Previo a la selección de sitios, se construye una tabla que indica qué taxones pueden ser diferenciados entre si. La relación entre dos taxones puede ser:

0) Irresolvible: Los dos taxones no presentan ningún sitio sin superposición de valores
1) Parcialmente resolvibles: Los taxones presentan sitios de **tipo 2** con valores que permiten distinguirlos del resto
2) Completamente resolvibles: Por lo menos uno de los taxones presenta sitios de **tipo 1** (todo taxón con sitios de este tipo debería ser resolvible con respecto a todos los demás taxones)

La tabla producida no es simétrica con respecto a la diagonal, ya que las relaciones entre taxones no son necesariamente simétricas.

**NOTA**: Cuando un taxón es resoluble con respecto a sí mismo, significa que tiene valores faltantes (distintos de si mismos)

In [169]:
# Get solvable taxa
solvable_full = np.full((len(orders), len(orders)), False)
solvable_partial = np.full((len(orders), len(orders)), False)
for idx, tax in enumerate(orders_sites):
    for idx2, tax2 in enumerate(orders_sites):
        solvable_full[idx, idx2] = ((tax & tax2).sum(axis = 1) == 0).sum() > 0
        solvable_partial[idx, idx2] = ((tax & ~tax2).sum(axis=1) > 0).sum() > 0
solvable = solvable_full.astype(int) + solvable_partial.astype(int)

solvable = pd.DataFrame(solvable, index=orders, columns=orders)

# Function to apply styles
def color_true_false(value):
    color_keys = ['color: white; background-color: red',
                  'color: white; background-color: orange',
                  'color: white; background-color: green']
    return color_keys[value]

solvable = solvable.style.applymap(color_true_false)
solvable

,6236,6329,27838,46000,57287,70219,70232,70242,73930,114861,211184,211375,373738,1150982,2044726,2044973
6236,0,1,1,1,1,1,1,1,1,1,1,1,2,2,1,1
6329,1,0,2,1,1,1,1,1,1,1,1,1,2,2,2,1
27838,0,2,0,1,1,2,1,1,2,2,1,2,2,2,2,2
46000,1,1,1,0,1,1,1,1,1,1,1,1,2,2,2,1
57287,1,1,1,1,0,1,1,1,1,1,1,1,2,2,2,1
70219,1,1,2,1,1,0,1,1,1,1,1,1,2,2,2,1
70232,1,1,1,1,1,1,0,1,1,1,1,1,2,2,2,1
70242,1,1,1,1,1,1,1,0,1,1,1,1,2,2,2,2
73930,1,1,2,1,1,1,1,1,0,1,1,1,2,2,2,1
114861,1,1,2,1,1,1,1,1,1,0,1,2,2,2,2,1


In [ ]:
# build a signal-noise matrix
n_orders = orders_sites.shape[0]
signal_noise = np.zeros((n_orders, n_orders))

# calculate signal - noise for each taxon
ord_idxs = np.arange(n_orders)
for idx, ord in enumerate(orders_sites):
    comp_orders = np.full(n_orders, True)
    comp_orders[idx] = False
    comp_orders = orders_sites[comp_orders]
    
    # get high signal sites
    shared_signal = (ord & comp_orders).sum(axis=0) > 0
    high_signal = ((ord & ~comp_orders).sum(axis=0) > 0) & ~shared_signal
    #for _ in np.arange(comp_orders.shape[1]):
        
    break

In [ ]:
high_signal.sum(axis=1)

In [ ]:
shared_signal.sum(axis=1)

In [ ]:
ord[2]

In [ ]:
comp_orders[:,2]

In [ ]:
# count shared values per site
tax_layer = shared_matrix[0]

solved = np.array([], dtype=int)
unsolved = np.arange(tax_layer.shape[0])

selected_sites = []
for _ in np.arange(tax_layer.shape[1]):
    # count shared values for both solved and unsolved taxa, sort both counts by shared values in unsolved
    shared_vals = tax_layer[unsolved].sum(axis=0)
    shared_solved = tax_layer[solved].sum(axis=0)
    shared_sorted_idx = np.argsort(shared_vals)
    
    # clear already selected sites
    shared_sorted_idx = shared_sorted_idx[~np.isin(shared_sorted_idx, selected_sites)]
    
    # sort shared unsolved and solved sites
    shared_sorted = shared_vals[shared_sorted_idx]
    shared_solved = shared_solved[shared_sorted_idx]
    
    # remove all shared columns, break if remaining columns are all shared
    not_all_shared = shared_vals[shared_sorted_idx] != len(unsolved)
    shared_sorted = shared_sorted[not_all_shared]
    shared_solved = shared_solved[not_all_shared]
    shared_sorted_idx = shared_sorted_idx[not_all_shared]
    if not_all_shared.sum() == 0:
        print(f'Couldn\'t solve taxon, {len(unsolved)} unsolved taxa remaining')
    
    # from the sites with the least shared observations in unsolved taxa, select the one with the least shared observations in solved taxa
    min_obs_idx = shared_sorted == shared_sorted.min()
    shared_sorted = shared_sorted[min_obs_idx]
    shared_solved = shared_solved[min_obs_idx]
    shared_sorted_idx = shared_sorted_idx[min_obs_idx]

    # select the site
    selected_idx = shared_sorted_idx[np.argmin(shared_solved)]
    selected_sites.append(selected_idx)
    
    # update solved & unsolved taxa
    new_solved = unsolved[~tax_layer[unsolved, selected_idx]]
    solved = np.concatenate((solved, new_solved))
    unsolved = unsolved[tax_layer[unsolved, selected_idx]]

    if len(unsolved) == 0:
        print(f'Solved taxon with {len(selected_sites)} sites!')

In [ ]:
tax_layer[unsolved]